# Transformer Block

The Transformer block is a key component of the Transformer model, which was introduced in the paper "Attention is All You Need" by Vaswani et al.

The Transformer block consists of two main parts:

1. **Multi-Head Self-Attention Mechanism**: This mechanism allows the model to focus on different positions of the input sequence when producing the output. It does this by applying the attention mechanism multiple times in parallel (hence "multi-head") to the input. The attention scores are calculated using the formula:

    $$ \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V $$

    where $Q$, $K$, and $V$ are the query, key, and value vectors, and $d_k$ is the dimension of the key vectors.

2. **Position-wise Feed-Forward Networks**: These are fully connected feed-forward networks applied to each position separately and identically. This consists of two linear transformations with a ReLU activation in between.

The output of each sub-layer (Multi-Head Attention and Feed-Forward) is then passed through a residual connection followed by layer normalization.

The Transformer block can be visualized as follows:

<div style="text-align: center;">
    <img src="imgs/transformer.png" alt="Transformer Block Diagram" width="300">
</div>

This architecture allows the Transformer to handle dependencies regardless of their distance in the input or output sequences, making it effective for a wide range of tasks.



In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
import lightning as pl
from torch.utils.data import DataLoader, Dataset
from torchtext.datasets import Multi30k
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from torchmetrics.text import BLEUScore  # Use torchmetrics for BLEU score
import spacy
import os
import random
import numpy as np

# import glob  # No longer needed after removing old cleanup
import tarfile  # Add tarfile for manual extraction

# Import specific callbacks from lightning
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping

# Import the Transformer model components from our implementation

/home/giangson/anaconda3/envs/nlp_poetry/lib/python3.11/site-packages/torchtext/datasets/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/home/giangson/anaconda3/envs/nlp_poetry/lib/python3.11/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/home/giangson/anaconda3/envs/nlp_poetry/lib/python3.11/site-packages/torchtext/vocab/


## Setup and Configuration

### Seed for Reproducibility
We set random seeds for Python's `random`, `numpy`, and `torch` to ensure that our results are reproducible.



### Special tokens and Indices

Define special tokens used in sequence processing:
- `<pad>`: Padding token.
- `<sos>`: Start of Sentence token.
- `<eos>`: End of Sentence token.
- `<unk>`: Unknown token (for words not in the vocabulary).

We also define their corresponding fixed indices, ensuring `<pad>` is at index 0, which is often required by loss functions like `CrossEntropyLoss(ignore_index=...)`.



In [ ]:

# Define special tokens and indices
PAD_TOKEN = "<pad>"
SOS_TOKEN = "<sos>"
EOS_TOKEN = "<eos>"
UNK_TOKEN = "<unk>"
PAD_IDX = 0
SOS_IDX = 1
EOS_IDX = 2
UNK_IDX = 3
SPECIAL_TOKENS = [PAD_TOKEN, SOS_TOKEN, EOS_TOKEN, UNK_TOKEN]


## Data Handling with PyTorch Lightning DataModule

We define a `LightningDataModule` to encapsulate all data-related steps: downloading, tokenizing, building vocabulary, processing, and creating DataLoaders. This keeps the main training script clean and organized.

### `TranslationDataset`
A simple custom `Dataset` class to hold source and target tensor pairs.
#%%

In [ ]:

class TranslationDataset(Dataset):
    def __init__(self, src_data, tgt_data):
        self.src_data = src_data
        self.tgt_data = tgt_data
        
    def __len__(self):
        return len(self.src_data)
    
    def __getitem__(self, idx):
        src_item = self.src_data[idx]
        tgt_item = self.tgt_data[idx]
        return src_item, tgt_item


### `TransformerDataModule`
This class handles:
- **Initialization**: Sets batch size and data directory.
- **Spacy Models**: Loads English and German `spaCy` models for tokenization, downloading them if necessary.
- **Tokenization**: Defines `tokenize_de` and `tokenize_en` methods.
- **`prepare_data`**: Downloads the Multi30k dataset and ensures spaCy models are ready. This runs only once.
- **`setup`**: Builds vocabularies using `torchtext.vocab.build_vocab_from_iterator` and processes the raw data into tensors. This runs on every GPU/process in distributed settings.
- **`_process_data`**: Converts text pairs into sequences of indices, adding `<sos>` and `<eos>` tokens to the target sequence.
- **`_collate_fn`**: Pads sequences within each batch to the maximum length in that batch using `pad_sequence`. Ensures `batch_first=True`.
- **Dataloaders**: Defines `train_dataloader`, `val_dataloader`, and `test_dataloader` methods to provide data batches to the trainer. Uses multiple workers for potentially faster loading.



In [4]:
#%%
class TransformerDataModule(pl.LightningDataModule):
    """LightningDataModule for Multi30k dataset."""

    def __init__(self, batch_size: int = 32, data_dir: str = ".data"):
        super().__init__()
        self.batch_size = batch_size
        self.data_dir = data_dir
        self.spacy_en = None
        self.spacy_de = None
        self.de_vocab = None
        self.en_vocab = None
        self.train_data = None
        self.valid_data = None
        self.test_data = None

    def _load_spacy_models(self):
        """Loads spaCy models, downloading if necessary."""
        try:
            self.spacy_de = spacy.load("de_core_news_sm")
            self.spacy_en = spacy.load("en_core_web_sm")
        except OSError:
            print("Downloading spaCy models...")
            os.system("python -m spacy download de_core_news_sm")
            os.system("python -m spacy download en_core_web_sm")
            self.spacy_de = spacy.load("de_core_news_sm")
            self.spacy_en = spacy.load("en_core_web_sm")

    def tokenize_de(self, text):
        """Tokenize German text."""
        if self.spacy_de is None:
            self._load_spacy_models()  # Ensure loaded
        return [tok.text for tok in self.spacy_de.tokenizer(text.lower())]

    def tokenize_en(self, text):
        """Tokenize English text."""
        if self.spacy_en is None:
            self._load_spacy_models()  # Ensure loaded
        return [tok.text for tok in self.spacy_en.tokenizer(text.lower())]

    def _yield_tokens(self, data_iter, tokenizer):
        """Helper function for build_vocab_from_iterator."""
        for text in data_iter:
            yield tokenizer(text)

    def prepare_data(self):
        """Download dataset and spaCy models, manually extracting test set to avoid corruption. Runs once."""
        # Remove the previous cleanup logic as it was ineffective before download

        # Original prepare_data logic:
        print("Proceeding with Multi30k download...")
        # This will download train, valid, test archives if not present
        Multi30k(
            root=self.data_dir,
            split=(
                "train",
                "valid",
                "test",
            ),  # Need to request test split to ensure archive is downloaded
            language_pair=("de", "en"),
        )

        # --- Manually extract test.de and test.en to ensure correctness ---
        # Corrected path based on list_dir results
        data_root_dir = os.path.join(self.data_dir, "datasets", "Multi30k")
        test_archive_name = "mmt16_task1_test.tar.gz"
        test_archive_path = os.path.join(data_root_dir, test_archive_name)
        correct_test_files = ["test.de", "test.en"]

        if os.path.exists(test_archive_path):
            print(
                f"Manually extracting required test files from {test_archive_path}..."
            )
            try:
                with tarfile.open(test_archive_path, "r:gz") as tar:
                    extracted_count = 0
                    for member in tar.getmembers():
                        print(
                            f"  Found member in archive: {member.name} (Is file: {member.isfile()})"
                        )  # Log every member found
                        # Check if the member is one of the files we want
                        # Compare basename to handle potential leading './' in archive paths
                        member_basename = os.path.basename(member.name)
                        if (
                            member_basename in correct_test_files and member.isfile()
                        ):  # Also ensure it's a file
                            # Extract it directly into the data_root_dir, overwriting if necessary
                            member.name = member_basename  # Ensure it extracts flatly in data_root_dir
                            tar.extract(member, path=data_root_dir)
                            print(f"  Extracted: {member.name}")
                            extracted_count += 1
                    if extracted_count == len(correct_test_files):
                        print(
                            f"Successfully extracted {extracted_count} required test files."
                        )
                    else:
                        print(
                            f"Warning: Expected {len(correct_test_files)} test files, but extracted {extracted_count}. Check archive content."
                        )
            except tarfile.TarError as e:
                print(
                    f"Error extracting {test_archive_path}: {e}. Manual cleanup might be needed."
                )
            except Exception as e:
                print(f"An unexpected error occurred during manual extraction: {e}")
        else:
            print(
                f"Test archive {test_archive_path} not found after download attempt. Cannot manually extract."
            )
        # --- End manual extraction ---

        print("Loading spaCy models...")
        self._load_spacy_models()  # Ensure models are available
        print("prepare_data() complete.")

    def setup(self, stage: str = None):
        """Build vocabularies and process datasets. Runs on each process."""
        # Load iterators
        train_iter, valid_iter, test_iter = Multi30k(
            root=self.data_dir,
            split=("train", "valid", "test"),
            language_pair=("de", "en"),
        )
        # Load train_iter fully for vocab building (might be memory intensive for huge datasets)
        train_list = list(train_iter)

        # Build vocabularies only if they haven't been built
        # Ensure tokenizers are ready before building vocab
        if self.spacy_de is None or self.spacy_en is None:
            self._load_spacy_models()

        if self.de_vocab is None:
            print("Building German vocabulary...")
            self.de_vocab = build_vocab_from_iterator(
                self._yield_tokens((src for src, _ in train_list), self.tokenize_de),
                min_freq=2,  # Ignore rare words
                specials=SPECIAL_TOKENS,
                special_first=True,
            )
            self.de_vocab.set_default_index(
                UNK_IDX
            )  # Set default for out-of-vocabulary words

        if self.en_vocab is None:
            print("Building English vocabulary...")
            self.en_vocab = build_vocab_from_iterator(
                self._yield_tokens((tgt for _, tgt in train_list), self.tokenize_en),
                min_freq=2,
                specials=SPECIAL_TOKENS,
                special_first=True,
            )
            self.en_vocab.set_default_index(UNK_IDX)

        print("Processing datasets...")
        # Process data (convert text to indices)
        train_src, train_tgt = self._process_data(train_list)
        valid_src, valid_tgt = self._process_data(list(valid_iter))
        print("Attempting to process test_iter...")  # Log before processing test_iter
        test_src, test_tgt = self._process_data(list(test_iter))

        # Create datasets
        self.train_data = TranslationDataset(train_src, train_tgt)
        self.valid_data = TranslationDataset(valid_src, valid_tgt)
        self.test_data = TranslationDataset(test_src, test_tgt)
        print("Data setup complete.")

    def _process_data(self, data):
        """Process raw data pairs into tokenized and indexed tensors."""
        src_tensors = []
        tgt_tensors = []
        for src_text, tgt_text in data:
            src_tokens = self.tokenize_de(src_text)
            tgt_tokens = self.tokenize_en(tgt_text)

            src_indices = [self.de_vocab[token] for token in src_tokens]
            # Add SOS/EOS to target sequence only
            tgt_indices = (
                [SOS_IDX] + [self.en_vocab[token] for token in tgt_tokens] + [EOS_IDX]
            )

            src_tensors.append(torch.tensor(src_indices, dtype=torch.long))
            tgt_tensors.append(torch.tensor(tgt_indices, dtype=torch.long))
        return src_tensors, tgt_tensors

    def _collate_fn(self, batch):
        """Collate function to pad sequences within a batch."""
        src_batch, tgt_batch = [], []
        for src_sample, tgt_sample in batch:  # Corrected variable names
            src_batch.append(src_sample)
            tgt_batch.append(tgt_sample)

        # Pad sequences to the max length in the current batch
        src_batch = pad_sequence(
            src_batch, padding_value=self.de_vocab[PAD_TOKEN], batch_first=True
        )
        tgt_batch = pad_sequence(
            tgt_batch, padding_value=self.en_vocab[PAD_TOKEN], batch_first=True
        )
        return src_batch, tgt_batch

    def train_dataloader(self):
        """Returns the DataLoader for the training set."""
        if self.train_data is None:
            self.setup()  # Ensure setup has run
        return DataLoader(
            self.train_data,
            batch_size=self.batch_size,
            shuffle=True,
            collate_fn=self._collate_fn,
            pin_memory=True,  # May speed up GPU training
        )

    def val_dataloader(self):
        """Returns the DataLoader for the validation set."""
        if self.valid_data is None:
            self.setup()
        return DataLoader(
            self.valid_data,
            batch_size=self.batch_size,
            shuffle=False,
            collate_fn=self._collate_fn,
            pin_memory=True,
        )

    def test_dataloader(self):
        """Returns the DataLoader for the test set."""
        if self.test_data is None:
            self.setup()
        return DataLoader(
            self.test_data,
            batch_size=self.batch_size,
            shuffle=False,
            collate_fn=self._collate_fn,
            pin_memory=True,
        )


## Transformer Model with PyTorch Lightning Module

### `TransformerLightningModule`
This class defines the core Transformer model logic within the PyTorch Lightning framework.
- **Initialization**:
    - Takes vocabulary sizes, model hyperparameters (d_model, heads, layers, d_ff, dropout, etc.), learning rate, and special token indices.
    - Initializes the `Transformer` model (imported from `transformer_from_scratch`).
    - Initializes the loss function (`CrossEntropyLoss`), ignoring the padding index.
    - Initializes `BLEUScore` metrics for validation and testing.
    - Stores references to vocabularies and tokenizers needed for BLEU calculation and translation.
    - Uses `save_hyperparameters` to automatically log hyperparameters.
- **`forward`**: Defines the forward pass, simply calling the internal `transformer` model. The `transformer`'s forward method handles mask creation internally.
- **`_calculate_loss`**: A helper method to compute and log the loss for any step (train, val, test). It correctly prepares the target sequences for input (`tgt_input`) and loss calculation (`tgt_output_expected`) based on the standard teacher-forcing approach.
- **`training_step`, `validation_step`, `test_step`**: Implement the standard Lightning steps, calling `_calculate_loss`. The validation and test steps also call `_update_bleu` to accumulate predictions and targets for BLEU score calculation.
- **`_update_bleu`**: Calculates predicted sentences (using greedy decoding for simplicity during evaluation steps) and target sentences from a batch, converting indices back to text and handling special tokens. Updates the corresponding BLEU metric (`val_bleu` or `test_bleu`).
- **`on_validation_epoch_end`, `on_test_epoch_end`**: Compute the final BLEU score for the epoch using the accumulated metric state, log it, and reset the metric for the next epoch/run.
- **`configure_optimizers`**: Sets up the Adam optimizer and a `ReduceLROnPlateau` learning rate scheduler that reduces the learning rate if the validation loss plateaus.
- **`translate_sentence`**: Provides a method to translate a single German sentence into English using the trained model. It handles tokenization, indexing, running the model autoregressively (feeding predictions back as input), and converting the output indices back to a readable sentence.


In [6]:
from transformer_from_scratch import Transformer


class TransformerLightningModule(pl.LightningModule):
    """LightningModule for the Transformer model."""

    def __init__(
        self,
        src_vocab_size: int,
        tgt_vocab_size: int,
        d_model: int = 256,
        num_heads: int = 8,
        num_layers: int = 3,
        d_ff: int = 512,
        max_seq_length: int = 100,
        dropout: float = 0.1,
        learning_rate: float = 0.0005,
        pad_idx: int = PAD_IDX,
        sos_idx: int = SOS_IDX,
        eos_idx: int = EOS_IDX,
        de_vocab=None,  # Pass vocab/tokenizer for translation & BLEU
        en_vocab=None,
        tokenize_de=None,
    ):
        super().__init__()
        # Store hyperparameters automatically, ignore non-primitive types for logging
        self.save_hyperparameters(ignore=["de_vocab", "en_vocab", "tokenize_de"])

        # Ensure pad_idx is passed correctly if needed by Transformer sub-modules
        # Assuming Transformer internal components get pad_idx from somewhere else or don't need it explicitly in init
        self.transformer = Transformer(
            src_vocab_size=src_vocab_size,
            tgt_vocab_size=tgt_vocab_size,
            d_model=d_model,
            num_heads=num_heads,
            num_layers=num_layers,
            d_ff=d_ff,
            max_seq_length=max_seq_length,
            dropout=dropout,
        )
        self.criterion = nn.CrossEntropyLoss(ignore_index=self.hparams.pad_idx)
        # Access learning rate via self.hparams
        # self.learning_rate = learning_rate (already saved in hparams)

        # Store necessary components passed for BLEU/translation
        # Using _ prefix to indicate they are primarily for internal use within the module
        self._de_vocab = de_vocab
        self._en_vocab = en_vocab
        self._tokenize_de = tokenize_de

        # Initialize BLEU score metrics
        self.val_bleu = BLEUScore()
        self.test_bleu = BLEUScore()
        
    def forward(self, src, tgt):
        """Forward pass through the Transformer model."""
        return self.transformer(src, tgt)
    
    def _calculate_loss(self, batch, stage="train"):
        """Helper function to calculate and log loss."""
        src, tgt = batch
        # Prepare target sequences:
        # Input to decoder: Starts with SOS, ends before EOS
        # Target for loss: Starts after SOS, ends with EOS
        tgt_input = tgt[:, :-1]
        tgt_output_expected = tgt[:, 1:]

        # Get model prediction
        # The model's forward takes src and decoder input (tgt_input)
        output = self(
            src, tgt_input
        )  # Shape: (batch_size, tgt_len - 1, tgt_vocab_size)

        # Reshape for CrossEntropyLoss (expects N, C)
        # N = batch_size * sequence_length, C = vocab_size
        output_dim = output.shape[-1]
        output_flat = output.contiguous().view(
            -1, output_dim
        )  # Shape: (batch_size * (tgt_len - 1), tgt_vocab_size)
        tgt_flat = tgt_output_expected.contiguous().view(
            -1
        )  # Shape: (batch_size * (tgt_len - 1))

        loss = self.criterion(output_flat, tgt_flat)

        # Log the loss
        self.log(
            f"{stage}_loss",
            loss,
            on_step=(stage == "train"),
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )
        return loss, output  # Return raw output for BLEU calculation
    
    def _update_bleu(self, batch, model_output, bleu_metric):
        """Helper to calculate and update BLEU score for a batch."""
        if self._en_vocab is None:
            return  # Cannot calculate BLEU without vocab

        src, tgt = batch
        tgt_output_expected = tgt[:, 1:]  # Target for comparison (batch_size, seq_len)

        # Get predicted token indices (greedy decoding)
        # model_output shape: (batch_size, seq_len, vocab_size)
        pred_indices = model_output.argmax(dim=-1)  # Shape: (batch_size, seq_len)

        # Convert indices to text sentences
        pred_sentences_text = []
        target_sentences_text = []

        itos = self._en_vocab.get_itos()  # Integer-to-string mapping

        for i in range(pred_indices.shape[0]):  # Iterate through batch
            # Predicted sentence
            pred_sent = []
            for idx in pred_indices[i].tolist():
                if idx == self.hparams.eos_idx:
                    break
                # Include UNK, but skip PAD and SOS
                if idx != self.hparams.pad_idx and idx != self.hparams.sos_idx:
                    pred_sent.append(itos[idx])
            pred_sentences_text.append(" ".join(pred_sent))

            # Target sentence (needs to be list of lists for BLEU score)
            target_sent = []
            for idx in tgt_output_expected[i].tolist():
                if idx == self.hparams.eos_idx:
                    break
                # Skip PAD and SOS
                if idx != self.hparams.pad_idx and idx != self.hparams.sos_idx:
                    target_sent.append(itos[idx])
            # BLEU score expects list of reference sentences
            target_sentences_text.append([" ".join(target_sent)])

        # Update the BLEU metric state
        if pred_sentences_text and target_sentences_text:  # Ensure not empty
            bleu_metric.update(pred_sentences_text, target_sentences_text)
    
    def training_step(self, batch, batch_idx):
        """Performs a single training step."""
        loss, _ = self._calculate_loss(batch, stage="train")
        return loss
    
    def validation_step(self, batch, batch_idx):
        """Performs a single validation step."""
        loss, output = self._calculate_loss(batch, stage="val")
        # Calculate BLEU score for this batch
        self._update_bleu(batch, output, self.val_bleu)
        # Loss is automatically logged by _calculate_loss
        return loss
    
    def test_step(self, batch, batch_idx):
        """Performs a single test step."""
        loss, output = self._calculate_loss(batch, stage="test")
        # Calculate BLEU score for this batch
        self._update_bleu(batch, output, self.test_bleu)
        # Loss is automatically logged by _calculate_loss
        return loss
    
    def on_validation_epoch_end(self):
        """Computes and logs validation BLEU score at the end of the epoch."""
        if self._en_vocab:  # Only compute if vocab is available
            bleu = self.val_bleu.compute()
            self.log("val_bleu", bleu, prog_bar=True)
            self.val_bleu.reset()  # Reset metric state for next epoch

    def on_test_epoch_end(self):
        """Computes and logs test BLEU score at the end of the test run."""
        if self._en_vocab:
            bleu = self.test_bleu.compute()
            self.log("test_bleu", bleu, prog_bar=True)
            self.test_bleu.reset()  # Reset metric state
            
    def configure_optimizers(self):
        """Configures the optimizer and learning rate scheduler."""
        optimizer = optim.Adam(self.parameters(), lr=self.hparams.learning_rate)
        # Learning rate scheduler: Reduces LR when validation loss plateaus
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode="min",  # Reduce LR when the monitored quantity stops decreasing
            factor=0.1,  # Factor by which the learning rate will be reduced
            patience=2,  # Number of epochs with no improvement after which LR will be reduced
            verbose=True,
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",  # Quantity to monitor
                "interval": "epoch",  # Check scheduler condition every epoch
                "frequency": 1,  # Check every 1 epoch
            },
        }
        
    def translate_sentence(self, sentence: str, max_len=50):
        """Translates a single source sentence using the trained model."""
        # Ensure necessary components are available
        if (
            self._de_vocab is None
            or self._en_vocab is None
            or self._tokenize_de is None
        ):
            raise RuntimeError(
                "Vocabularies and tokenizer must be available for translation. Ensure they are passed during init or loaded."
            )

        self.eval()  # Set the model to evaluation mode (disables dropout, etc.)

        # Tokenize and numericalize the source sentence
        tokens = self._tokenize_de(sentence)
        indices = [self._de_vocab[token] for token in tokens]
        # Add batch dimension and move to the correct device
        src_tensor = torch.LongTensor(indices).unsqueeze(0).to(self.device)

        # Initialize the target sequence with the SOS token
        tgt_tensor = torch.LongTensor([[self.hparams.sos_idx]]).to(
            self.device
        )  # Shape: (1, 1)

        # Autoregressive generation loop
        for _ in range(max_len):
            with torch.no_grad():  # No need to track gradients during inference
                # Pass current source and target sequence to the model
                # The model's forward pass will handle mask creation
                output = self.transformer(
                    src_tensor, tgt_tensor
                )  # Output shape: (1, current_tgt_len, tgt_vocab_size)

            # Get the predicted token index for the *last* position in the sequence
            pred_token_idx = output.argmax(2)[:, -1].item()

            # Append the predicted token index to the target sequence
            # Shape becomes (1, current_tgt_len + 1)
            tgt_tensor = torch.cat(
                [tgt_tensor, torch.LongTensor([[pred_token_idx]]).to(self.device)],
                dim=1,
            )

            # Stop generation if the EOS token is predicted
            if pred_token_idx == self.hparams.eos_idx:
                break

        # Convert the generated target indices back to tokens
        # Skip the initial SOS token ([1:])
        tgt_indices = tgt_tensor.squeeze(0).tolist()[1:]
        tgt_tokens = []
        itos = self._en_vocab.get_itos()
        for idx in tgt_indices:
            if idx == self.hparams.eos_idx:  # Stop if EOS is encountered
                break
            tgt_tokens.append(itos[idx])

        return " ".join(tgt_tokens)

In [7]:
config = {
    "batch_size": 64,  # Adjusted batch size
    "d_model": 256,
    "num_heads": 8,
    "num_layers": 3,
    "d_ff": 512,
    "max_seq_length": 100,  # Should match model capacity
    "dropout": 0.1,
    "learning_rate": 0.0005,
    "num_epochs": 30,
    "patience": 5,  # Early stopping patience
    "data_dir": ".data",  # Directory for dataset download
    "checkpoint_dir": "checkpoints_transformer",  # Directory for saving models
}


# 🧠 Giải thích cấu hình `config = {...}` từng dòng

| Tham số               | Giá trị                         | Giải thích                                                                 |
|-----------------------|----------------------------------|-----------------------------------------------------------------------------|
| `batch_size`          | 64                               | Mỗi lần huấn luyện, mô hình xử lý 64 cặp câu (nguồn + đích).              |
| `d_model`             | 256                              | Mỗi từ được biểu diễn bằng vector 256 chiều (embedding + transformer layers). |
| `num_heads`           | 8                                | Multi-head attention có 8 đầu để học các quan hệ khác nhau giữa các từ.   |
| `num_layers`          | 3                                | Gồm 3 encoder layer + 3 decoder layer.                                     |
| `d_ff`                | 512                              | Feed-forward network gồm 2 lớp, mỗi lớp có 512 neuron.                     |
| `max_seq_length`      | 100                              | Chiều dài tối đa của câu. Nếu ngắn hơn thì padding, dài hơn thì bị cắt.    |
| `dropout`             | 0.1                              | Loại ngẫu nhiên 10% neuron để tránh overfitting trong lúc huấn luyện.     |
| `learning_rate`       | 0.0005                           | Tốc độ học. Bước nhảy nhỏ giúp mô hình học chậm mà chắc.                  |
| `num_epochs`          | 30                               | Số lần học qua toàn bộ dữ liệu. Tối đa 30 lần.                             |
| `patience`            | 5                                | Early stopping. Nếu validation không cải thiện sau 5 epoch thì dừng sớm.  |
| `data_dir`            | ".data"                          | Thư mục chứa dữ liệu dịch như IWSLT hoặc Multi30K.                         |
| `checkpoint_dir`      | "checkpoints_transformer"        | Thư mục lưu mô hình tốt nhất dựa trên `val_loss`.                         |

# 📦 Ví dụ minh họa

Giả sử bạn đang dịch:

> "Ich liebe dich" → "I love you"

Thì các thông số ảnh hưởng như sau:

* `d_model`: 256 → "Ich" được chuyển thành vector 256 chiều  
* `num_heads`: 8 → Mô hình có 8 góc nhìn để hiểu mối quan hệ giữa "liebe", "Ich", "dich"  
* `max_seq_length`: 100 → Nếu câu dài hơn 100 từ thì sẽ bị cắt  
* `dropout`: 0.1 → Ngẫu nhiên tắt bớt neuron để tránh học lệch  
* `learning_rate`: 0.0005 → Mô hình học từ từ để tối ưu chính xác

# 💡 Gợi ý cấu hình mở rộng (tuỳ chọn)

* `label_smoothing`: 0.1  
  → Giảm độ tự tin quá mức khi training với CrossEntropyLoss

* `warmup_steps`: 4000  
  → Dùng khi áp dụng Noam Scheduler từ paper gốc

* `gradient_clip_val`: 1.0  
  → Tránh hiện tượng gradient exploding khi huấn luyện

* `save_top_k`: 1  
  → Chỉ lưu mô hình tốt nhất

* `monitor`: "val_loss"  
  → Dựa vào chỉ số này để quyết định khi nào lưu mô hình



In [8]:
datamodule = TransformerDataModule(
    batch_size=config["batch_size"], data_dir=config["data_dir"]

)

# 🧠 Giải thích từng dòng dành cho người mới bắt đầu

## 1. `TransformerDataModule(...)`

| Thành phần                    | Mô tả chi tiết                                                                 |
|------------------------------|----------------------------------------------------------------------------------|
| **Là gì?**                   | Một **DataModule** theo chuẩn của **PyTorch Lightning**                         |
| **Chức năng**                | - Quản lý việc tải, xử lý, chia tập `train`, `val`, `test` từ dữ liệu đầu vào   |
|                              | - Tạo ra các `DataLoader` chuẩn để phục vụ mô hình Transformer                  |
| **Ưu điểm**                  | Định nghĩa một lần duy nhất, sau đó sử dụng lại ở bất kỳ giai đoạn nào          |
|                              | như huấn luyện, kiểm thử, validate, hoặc dịch câu                                |
| **Tích hợp**                 | Tương thích với Lightning, giúp quản lý dữ liệu xuyên suốt pipeline huấn luyện  |

🎯 **Mục tiêu:**  
Giúp bạn không phải viết lại code xử lý dữ liệu cho từng giai đoạn. Chỉ cần khởi tạo 1 lần là dùng được mọi nơi.

---

## 2. 🔢 Tham số truyền vào

| Tham số       | Ý nghĩa                                                                                        |
|---------------|------------------------------------------------------------------------------------------------|
| `batch_size`  | Số lượng cặp câu (nguồn + đích) được xử lý mỗi lần training hoặc validation. Lấy từ `config`. |
| `data_dir`    | Thư mục chứa dữ liệu dịch: bao gồm file `raw`, tokenizer, vocabulary…                         |

💡 **Mục đích:**  
Giúp mô hình tự động quản lý mọi thứ liên quan đến dữ liệu đầu vào, bạn không cần truyền thủ công cho từng `DataLoader`.



In [ ]:
# We prepare the data module to ensure the vocabularies are built
datamodule.prepare_data()


Proceeding with Multi30k download...


/home/giangson/anaconda3/envs/nlp_poetry/lib/python3.11/site-packages/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


Manually extracting required test files from .data/datasets/Multi30k/mmt16_task1_test.tar.gz...
  Found member in archive: ./._test.de (Is file: True)
  Found member in archive: ./test.de (Is file: True)
  Extracted: test.de
  Found member in archive: ./._test.en (Is file: True)
  Found member in archive: ./test.en (Is file: True)
  Extracted: test.en
  Found member in archive: ./._test.fr (Is file: True)
  Found member in archive: ./test.fr (Is file: True)
Successfully extracted 2 required test files.
Loading spaCy models...
prepare_data() complete.


# 🧠 Giải thích dòng `datamodule.prepare_data()`

| Thành phần                 | Giải thích chi tiết                                                                 |
|---------------------------|--------------------------------------------------------------------------------------|
| **Mục tiêu chính**        | Chuẩn bị dữ liệu đầu vào cho mô hình dịch bằng Transformer                          |
| **Tải dữ liệu**           | Tải + giải nén dataset như Multi30k, IWSLT nếu chưa có trong thư mục                |
| **Tiền xử lý câu**        | Chuẩn hóa, chuyển về chữ thường (`lowercase`), loại ký tự đặc biệt, làm sạch câu    |
| **Tokenizer**             | Tách câu thành từng token hoặc subword (dùng Spacy, BPE, SentencePiece...)          |
| **Tạo vocabulary**        | Ánh xạ mỗi từ về một chỉ số (`"hello"` → 482, `"world"` → 203)                      |
| **Lưu vocab ra file**     | Lưu file vocab để tái sử dụng lần sau, tránh việc tạo lại mỗi lần chạy              |
| **Ràng buộc bắt buộc**    | Nếu không gọi `prepare_data()`, khi huấn luyện sẽ lỗi: `"vocab is None"`            |

---

## 🎯 Tóm tắt lợi ích

* Giúp xử lý và chuẩn hóa toàn bộ dữ liệu ngôn ngữ.
* Tạo token và vocab cho mô hình sử dụng sau này.
* Là bước bắt buộc để mô hình có thể huấn luyện, dự đoán, hoặc đánh giá.

---

## 💡 Sơ đồ minh họa trực quan

          ┌────────────────────┐
          │ Raw Data (en-de)   │
          └────────┬───────────┘
                   ▼
        ┌──────────────────────┐
        │ prepare_data()       │
        │                      │
        │  ✔ Load + clean data │
        │  ✔ Tokenize          │
        │  ✔ Build vocab       │
        └────────┬─────────────┘
                 ▼
      ┌───────────────────────────┐
      │  self.de_vocab, en_vocab  │
      │  self.train_dataloader()  │
      └───────────────────────────┘


In [ ]:

# Setup the model so that the vocabularies are built
datamodule.setup()


/home/giangson/anaconda3/envs/nlp_poetry/lib/python3.11/site-packages/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


Building German vocabulary...
Building English vocabulary...
Processing datasets...
Attempting to process test_iter...
Data setup complete.


# ✅ Giải thích chi tiết cho người mới học

## 1. Gọi `datamodule.prepare_data()`

* Dữ liệu sẽ được tải về (Multi30k, IWSLT, ...).
* Thực hiện xử lý thô: chuẩn hóa, lowercase, loại bỏ ký tự lạ.
* Tokenize các câu bằng tokenizer (Spacy, BPE...).
* Tạo vocabulary: ánh xạ mỗi token → chỉ số.
  - Ví dụ: `"hello"` → `482`, `"world"` → `203`
* Lưu lại các file vocab để dùng lại sau.
* ⚠️ Nếu bỏ qua bước này → lỗi `"vocab is None"` khi huấn luyện hoặc dịch câu.

---

## 2. Gọi `datamodule.setup()`

* Chia dữ liệu ra thành: `train`, `val`, `test`.
* Lưu các vocab đã tạo vào biến nội bộ như:
  - `datamodule.de_vocab`
  - `datamodule.en_vocab`
* Tạo sẵn các DataLoader:
  - `train_dataloader()`
  - `val_dataloader()`
  - `test_dataloader()`

🎯 **Mục đích:** chuẩn bị mọi thứ để mô hình có thể huấn luyện hoặc dịch câu.

---

## 🔁 So sánh `prepare_data()` và `setup()`

| Giai đoạn        | Chức năng chính                                      |
|------------------|------------------------------------------------------|
| `prepare_data()` | Tải dữ liệu + tiền xử lý + tạo tokenizer/vocab       |
| `setup()`        | Chia thành các tập `train/val/test` + chuẩn bị DataLoader |

---

## 🧩 Sơ đồ minh họa luồng dữ liệu

               ┌──────────────────────┐
               │  prepare_data()      │
               └─────────┬────────────┘
                         ▼
             ┌───────────────────────────┐
             │ Tokenizer + Vocabulary    │
             └─────────┬─────────────────┘
                       ▼
              ┌────────────────────┐
              │   setup()          │
              │ ┌───────────────┐ │
              │ │ Train Dataset │ │
              │ │ Val Dataset   │ │
              │ │ Test Dataset  │ │
              │ └───────────────┘ │
              └─────────┬────────┘
                        ▼
              ┌─────────────────────┐
              │ DataLoaders Ready   │
              └─────────────────────┘

---

## 🔚 Sau setup, bạn có thể:

* ✅ Dịch câu:

model._de_vocab = datamodule.de_vocab
model._en_vocab = datamodule.en_vocab
model._tokenize_de = datamodule.tokenize_de

* ✅ Huấn luyện mô hình:

trainer.fit(model, datamodule=datamodule)

In [10]:
# Get vocab sizes and special indices from the datamodule after setup
src_vocab_size = len(datamodule.de_vocab)
tgt_vocab_size = len(datamodule.en_vocab)
pad_idx = datamodule.de_vocab[PAD_TOKEN]
sos_idx = datamodule.en_vocab[SOS_TOKEN]
eos_idx = datamodule.en_vocab[EOS_TOKEN]

print(f"Source vocabulary size: {src_vocab_size}")
print(f"Target vocabulary size: {tgt_vocab_size}")
print(f"PAD index: {pad_idx}, SOS index: {sos_idx}, EOS index: {eos_idx}")
#%%

Source vocabulary size: 7853
Target vocabulary size: 5893
PAD index: 0, SOS index: 1, EOS index: 2


In [11]:
#%%
# Setup the model
model = TransformerLightningModule(
    src_vocab_size=src_vocab_size,
    tgt_vocab_size=tgt_vocab_size,
    d_model=config["d_model"],
    num_heads=config["num_heads"],
    num_layers=config["num_layers"],
    d_ff=config["d_ff"],
    max_seq_length=config["max_seq_length"],
    dropout=config["dropout"],
    learning_rate=config["learning_rate"],
    pad_idx=pad_idx,  # Pass the actual PAD index
    sos_idx=sos_idx,  # Pass the actual SOS index
    eos_idx=eos_idx,  # Pass the actual EOS index
    # Pass vocabs/tokenizer for BLEU calculation and translation method
    de_vocab=datamodule.de_vocab,
    en_vocab=datamodule.en_vocab,
    tokenize_de=datamodule.tokenize_de,
)
#%%

In [13]:
#%%
# Checkpoint callback: Saves the best model based on validation loss
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",  # Metric to monitor
    dirpath=config["checkpoint_dir"],  # Directory to save checkpoints
    filename="transformer-best-{epoch:02d}-{val_loss:.2f}-{val_bleu:.4f}",  # Filename format
    save_top_k=1,  # Save only the best model
    mode="min",  # Mode for the monitored metric (minimize loss)
    save_last=True,  # Optionally save the last checkpoint as well
)
# Early stopping callback: Stops training if validation loss doesn't improve
early_stop_callback = EarlyStopping(
    monitor="val_loss",  # Metric to monitor
    patience=config["patience"],  # Number of epochs to wait for improvement
    verbose=True,  # Print messages when stopping
    mode="min",  # Mode for the monitored metric
)

In [14]:
#%%
# Initialize the Trainer
trainer = pl.Trainer(
    max_epochs=config["num_epochs"],
    accelerator="auto",  # Automatically selects GPU/MPS/CPU
    devices=-1,  # Use all devices (can be >1 for multi-GPU)
    callbacks=[checkpoint_callback, early_stop_callback],  # List of callbacks
    log_every_n_steps=20,  # How often to log metrics
)
# Start the training process
print("\n--- Starting Training ---")
trainer.fit(model, datamodule=datamodule)  # Pass both model and datamodule

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



--- Starting Training ---
Proceeding with Multi30k download...
Manually extracting required test files from .data/datasets/Multi30k/mmt16_task1_test.tar.gz...
  Found member in archive: ./._test.de (Is file: True)
  Found member in archive: ./test.de (Is file: True)
  Extracted: test.de
  Found member in archive: ./._test.en (Is file: True)
  Found member in archive: ./test.en (Is file: True)
  Extracted: test.en
  Found member in archive: ./._test.fr (Is file: True)
  Found member in archive: ./test.fr (Is file: True)
Successfully extracted 2 required test files.
Loading spaCy models...
prepare_data() complete.
Processing datasets...
Attempting to process test_iter...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Data setup complete.



  | Name        | Type             | Params | Mode 
---------------------------------------------------------
0 | transformer | Transformer      | 9.0 M  | train
1 | criterion   | CrossEntropyLoss | 0      | train
2 | _de_vocab   | Vocab            | 0      | train
3 | _en_vocab   | Vocab            | 0      | train
4 | val_bleu    | BLEUScore        | 0      | train
5 | test_bleu   | BLEUScore        | 0      | train
---------------------------------------------------------
9.0 M     Trainable params
0         Non-trainable params
9.0 M     Total params
35.949    Total estimated model params size (MB)
134       Modules in train mode
0         Modules in eval mode


...

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved by 0.071 >= min_delta = 0.0. New best score: 1.775


Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved by 0.048 >= min_delta = 0.0. New best score: 1.727

Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [15]:
#%%
# Find the best checkpoint path saved by the callback
best_model_path = checkpoint_callback.best_model_path
if best_model_path and os.path.exists(best_model_path):
    print(f"Loading best model for testing from: {best_model_path}")
    # Run the test loop using the best checkpoint
    test_result = trainer.test(
        model, datamodule=datamodule, ckpt_path=best_model_path
    )
    print("Test Results:", test_result)
else:
    print(
        "Warning: No best model checkpoint found or path invalid. Testing with the model's current state (last epoch)."
    )
    # If no best model path, test with the model state after the last training epoch
    test_result = trainer.test(model, datamodule=datamodule)
    print("Test Results:", test_result)

Loading best model for testing from: /home/giangson/nlp/Chapter_06_Transformer_Architecture/checkpoints_transformer/transformer-best-epoch=05-val_loss=1.73-val_bleu=0.3321.ckpt
Proceeding with Multi30k download...
Manually extracting required test files from .data/datasets/Multi30k/mmt16_task1_test.tar.gz...
  Found member in archive: ./._test.de (Is file: True)
  Found member in archive: ./test.de (Is file: True)
  Extracted: test.de
  Found member in archive: ./._test.en (Is file: True)
  Found member in archive: ./test.en (Is file: True)
  Extracted: test.en
  Found member in archive: ./._test.fr (Is file: True)
  Found member in archive: ./test.fr (Is file: True)
Successfully extracted 2 required test files.
Loading spaCy models...
prepare_data() complete.
Processing datasets...
Attempting to process test_iter...


Restoring states from the checkpoint path at /home/giangson/nlp/Chapter_06_Transformer_Architecture/checkpoints_transformer/transformer-best-epoch=05-val_loss=1.73-val_bleu=0.3321.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Data setup complete.


Loaded model weights from the checkpoint at /home/giangson/nlp/Chapter_06_Transformer_Architecture/checkpoints_transformer/transformer-best-epoch=05-val_loss=1.73-val_bleu=0.3321.ckpt
/home/giangson/anaconda3/envs/nlp_poetry/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_bleu         │    0.3290563225746155     │
│         test_loss         │    1.7571927309036255     │
└───────────────────────────┴───────────────────────────┘

Test Results: [{'test_loss': 1.7571927309036255, 'test_bleu': 0.3290563225746155}]


In [16]:
#%%
# --- Translation Examples ---
print("\n--- Sample Translations (using best model) ---")
print("Loading the best model for translation...")
# Load the best model specifically for inference if needed, or use the one from trainer.test
final_model = model  # Use the model loaded/used by trainer.test
if best_model_path and os.path.exists(best_model_path):
    # Ensure the model loaded has the necessary vocabs etc. for translation
    final_model = TransformerLightningModule.load_from_checkpoint(
        best_model_path,
        # Need to provide args again if not saved in hparams, or if they were ignored
        de_vocab=datamodule.de_vocab,
        en_vocab=datamodule.en_vocab,
        tokenize_de=datamodule.tokenize_de,
    )
else:
    print("Using model's last state for final translations.")

print("Setting model to evaluation mode and moving to device...")
final_model.freeze()  # Set model to evaluation mode (important!)
# Ensure the model is on the correct device (especially if loading manually)
final_model.to(
    torch.device(
        "cuda"
        if torch.cuda.is_available()
        else ("mps" if torch.backends.mps.is_available() else "cpu")
    )
)

# Get a few samples from the original test data iterator
print("\nGenerating translations for test samples...")
_, _, test_iter_raw = Multi30k(
    root=config["data_dir"],
    split=("train", "valid", "test"),
    language_pair=("de", "en"),
)
test_samples_raw = list(test_iter_raw)[:5]

for src_text, tgt_text in test_samples_raw:
    # Use the translate_sentence method from our Lightning Module
    predicted_text = final_model.translate_sentence(src_text)
    print(f"Source:      {src_text}")
    print(f"Target:      {tgt_text}")
    print(f"Predicted:   {predicted_text}")
    print("-" * 20)

# Example with a sentence not from the dataset
print("\n--- Manual Translation Example ---")
manual_sentence = "Ein Mädchen in einem roten Kleid spielt auf einer Wiese."
print(f"Manual Source: {manual_sentence}")
print("Generating translation for manual sentence...")
predicted_translation = final_model.translate_sentence(manual_sentence)
print(f"Predicted:     {predicted_translation}")


--- Sample Translations (using best model) ---
Loading the best model for translation...
Setting model to evaluation mode and moving to device...

Generating translations for test samples...
Source:      Ein Mann mit einem orangefarbenen Hut, der etwas anstarrt.
Target:      A man in an orange hat starring at something.
Predicted:   a man in an orange hat is mixing something .
--------------------
Source:      Ein Boston Terrier läuft über saftig-grünes Gras vor einem weißen Zaun.
Target:      A Boston Terrier is running on lush green grass in front of a white fence.
Predicted:   a boston - haired female baseball player runs over a white fence .
--------------------
Source:      Ein Mädchen in einem Karateanzug bricht ein Brett mit einem Tritt.
Target:      A girl in karate uniform breaking a stick with a front kick.
Predicted:   a girl in a karate uniform is using a board with a toy .
--------------------
Source:      Fünf Leute in Winterjacken und mit Helmen stehen im Schnee mit Sch